# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import json
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import re
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [325]:
data = pd.read_json('/kaggle/input/convolve-epoch1/train.json',orient="index")
data.reset_index(inplace=True)
data.rename(columns={"index":"text"},inplace=True)
data.rename(columns={0:"target"},inplace=True)
target = data['target']
# data = Data.sample(50000)

In [326]:
len(data)

4152659

In [275]:
data.head()

,text,target
0,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...,normal
1,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...,normal
2,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...,normal
3,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...,normal
4,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...,normal


In [276]:
sample_log = data['text'].iloc[1000]
sample_log

' 1117838753 2005.06.03 R02-M1-N0-C:J12-U11 2005-06-03-15.45.53.478817 R02-M1-N0-C:J12-U11 RAS KERNEL INFO instruction cache parity error corrected\n'

In [362]:
def find_data(row):
    row = row.strip()
    regex = re.compile(r"""
                (\d{10,12}) #starting nums
                .*
                (\d{4}-\d{2}-\d{2}-\d{2}\.\d{2}\.\d{2}\.\d{6})
                \s
                ([A-Z0-9-:]{9,20}|NULL)
                \s(.*)
                """,re.VERBOSE)
    groups = regex.findall(row)
    if groups:
        return groups[0]
    return None

In [363]:
data["text_result"] = data["text"].apply(lambda x: find_data(x))

In [364]:
data["text_result"].isna().sum()/len(data)

0.006664886281295912

In [365]:
mismatch = data[data["text_result"].isna()]

In [366]:
mismatch["text"].iloc[10]

' 1119280406 2005.06.20 UNKNOWN_LOCATION 2005-06-20-08.13.26.687647 UNKNOWN_LOCATION NULL CMCS INFO Starting SystemController\n'

In [366]:
mismatch["text"].iloc[10]

' 1119280406 2005.06.20 UNKNOWN_LOCATION 2005-06-20-08.13.26.687647 UNKNOWN_LOCATION NULL CMCS INFO Starting SystemController\n'

In [367]:
mismatch["target"]

normal    27677
Name: target, dtype: int64

In [368]:
data.dropna(inplace = True)

In [369]:
data["text_result"][100000]

('1117989580',
 '2005-06-05-09.39.40.883980',
 'R30-M0-NA-C:J11-U01',
 'RAS KERNEL INFO generating core.1301')

In [371]:
data.head()

,text,target,text_result
0,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...,normal,"(1117838570, 2005-06-03-15.42.50.363779, R02-M..."
1,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...,normal,"(1117838570, 2005-06-03-15.42.50.527847, R02-M..."
2,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...,normal,"(1117838570, 2005-06-03-15.42.50.675872, R02-M..."
3,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...,normal,"(1117838570, 2005-06-03-15.42.50.823719, R02-M..."
4,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...,normal,"(1117838570, 2005-06-03-15.42.50.982731, R02-M..."


In [373]:
data["time"] = data["text_result"].apply(lambda x:x[1])
data["log_code"] = data["text_result"].apply(lambda x:x[2])
data["message"] = data["text_result"].apply(lambda x:x[3])

In [374]:
data.head()

,text,target,text_result,time,log_code,message
0,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...,normal,"(1117838570, 2005-06-03-15.42.50.363779, R02-M...",2005-06-03-15.42.50.363779,R02-M1-N0-C:J12-U11,RAS KERNEL INFO instruction cache parity error...
1,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...,normal,"(1117838570, 2005-06-03-15.42.50.527847, R02-M...",2005-06-03-15.42.50.527847,R02-M1-N0-C:J12-U11,RAS KERNEL INFO instruction cache parity error...
2,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...,normal,"(1117838570, 2005-06-03-15.42.50.675872, R02-M...",2005-06-03-15.42.50.675872,R02-M1-N0-C:J12-U11,RAS KERNEL INFO instruction cache parity error...
3,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...,normal,"(1117838570, 2005-06-03-15.42.50.823719, R02-M...",2005-06-03-15.42.50.823719,R02-M1-N0-C:J12-U11,RAS KERNEL INFO instruction cache parity error...
4,1117838570 2005.06.03 R02-M1-N0-C:J12-U11 200...,normal,"(1117838570, 2005-06-03-15.42.50.982731, R02-M...",2005-06-03-15.42.50.982731,R02-M1-N0-C:J12-U11,RAS KERNEL INFO instruction cache parity error...


In [375]:
data.to_csv("epoch1.csv", index = False)

In [376]:
data.drop(["text", "text_result"], axis=1, inplace = True)

In [377]:
data.columns

Index(['target', 'time', 'log_code', 'message'], dtype='object')

In [378]:
data.to_csv("epoch1_final.csv", index=False)

In [379]:
ls /kaggle/working

__notebook_source__.ipynb  epoch1.csv  epoch1_final.csv


In [ ]:
/kaggle/working/epoch1_final.csv